In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import os
import math
import pandas as pd
# database stuff:
import sqlalchemy
from sqlalchemy import create_engine, Column, String, Integer, Numeric, MetaData, Table, type_coerce, ForeignKey, case
from sqlalchemy.orm import mapper, create_session, relationship, aliased, Session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import case
from sqlalchemy.ext.automap import automap_base
import sqlalchemy.types as types
import numbers
from sqlalchemy.sql import and_, or_, not_, func
from sqlalchemy.sql import select
import glob
import gsw

%matplotlib inline


In [2]:
def ginterp(xval,xPeriod,yval,L,xlocs):
    # if not periodic, xPeriod=0
    fil=np.empty(np.size(xlocs))
    s=L/2.355
    for ii in range(0,xlocs.size):
        t=xlocs[ii]
        diff=[min(abs(x-t),abs(x-t+xPeriod), abs(x-t-xPeriod)) for x in xval]
        weight=[np.exp(-.5*x**2/s**2) if sum(diff<x)<2 or x < 5 else 0.0 for x in diff]
        #weight=[np.exp(-.5*x**2/s**2) for x in diff]
        weight=np.array(weight)
        if np.sum(weight)!=0:
            fil[ii]=np.sum(weight*yval)/np.sum(weight)
        else:
            fil[ii]=np.nan
    return(fil)

In [3]:
def ginterp2d(xval,xPeriod,yval,yPeriod,zval,L,M,zlocs_x,zlocs_y):
    # if not periodic, xPeriod=0
    #fil=np.empty(np.size(xlocs))
    s=L/2.355
    n=M/2.355
    sdict={}
    mat=np.empty((np.size(zlocs_x),np.size(zlocs_y)))
    for ii in range(0,zlocs_x.size):
        print(ii)
        for jj in range(0,zlocs_y.size):
            tx=zlocs_x[ii]
            ty=zlocs_y[jj]
            diffx=[min(abs(x-tx),abs(x-tx+xPeriod), abs(x-tx-xPeriod)) for x in xval]
            diffy=[min(abs(y-ty),abs(y-ty+yPeriod), abs(y-ty-yPeriod)) for y in yval]
            weight=[np.exp(-.5*(x**2+y**2)/(s**2+n**2)) if \
                    (sum(diffx<x)<3 or x < L) and (sum(diffy<y)<3 or y < M) \
                    else 0.0 for x, y in zip(diffx, diffy)]
            #weight=[np.exp(-.5*x**2/s**2) for x in diff]
            weight=np.array(weight)
            sdict[(tx,ty)]=np.sum(weight*zval)/np.sum(weight)
            mat[ii,jj]=np.sum(weight*zval)/np.sum(weight)
    return(sdict,mat)

In [4]:
#templates:
tW=nc.Dataset('/results/forcing/LiveOcean/boundary_conditions/LiveOcean_v201905_y2015m01d01.nc')
tN=nc.Dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/tracers/north/bioOBC_North_monthlySiNO3.nc')

In [5]:
newW = nc.Dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/tracers/west/testTurbW.nc', 'w', zlib=True)
newN = nc.Dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/tracers/north/testTurbN.nc', 'w', zlib=True)
#Copy dimensions
for dname, the_dim in tW.dimensions.items():
    newW.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
for dname, the_dim in tN.dimensions.items():
    newN.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)

In [6]:
tW.dimensions.keys()

dict_keys(['time_counter', 'deptht', 'yb', 'xbT'])

In [7]:
tN.dimensions.keys()

dict_keys(['deptht', 'time_counter', 'yb', 'xbT'])

In [8]:
tW.variables.keys()

dict_keys(['time_counter', 'deptht', 'yb', 'xbT', 'vosaline', 'votemper', 'NO3', 'Si', 'OXY', 'DIC', 'TA'])

In [9]:
tW.variables['NO3']

<class 'netCDF4._netCDF4.Variable'>
float64 NO3(time_counter, deptht, yb, xbT)
    _FillValue: nan
    grid: SalishSea2
    long_name: Nitrate
    units: muM
unlimited dimensions: time_counter
current shape = (1, 40, 1, 950)
filling on

In [10]:
tN.variables.keys()

dict_keys(['deptht', 'nav_lat', 'nav_lon', 'time_counter', 'NO3', 'Si'])

In [11]:
tN.variables['NO3']

<class 'netCDF4._netCDF4.Variable'>
float32 NO3(time_counter, deptht, yb, xbT)
    grid: SalishSea2
    units: muM
    long_name: Nitrate
unlimited dimensions: time_counter
current shape = (12, 40, 10, 30)
filling on, default _FillValue of 9.969209968386869e+36 used

In [12]:
# time_counter
time_counterN = newN.createVariable('time_counter', 'float32', ('time_counter'))
time_counterN[:]=[0.0]
time_counterW = newW.createVariable('time_counter', 'float32', ('time_counter'))
time_counterW[:]=[0.0]

In [13]:
#turConst
turConstN = newN.createVariable('turConst', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
turConstN.grid = tN.variables['NO3'].grid
turConstN.units = 'NTU'
turConstN.long_name = 'constant turbidity' 
turConstN[:]=1.0

turConstW = newW.createVariable('turConst', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
turConstW.grid = tW.variables['NO3'].grid
turConstW.units = 'NTU'
turConstW.long_name = 'constant turbidity' 
turConstW[:]=1.0

In [14]:
turConstW

<class 'netCDF4._netCDF4.Variable'>
float32 turConst(time_counter, deptht, yb, xbT)
    grid: SalishSea2
    units: NTU
    long_name: constant turbidity
unlimited dimensions: time_counter
current shape = (1, 40, 1, 950)
filling on, default _FillValue of 9.969209968386869e+36 used

In [15]:
turConstN

<class 'netCDF4._netCDF4.Variable'>
float32 turConst(time_counter, deptht, yb, xbT)
    grid: SalishSea2
    units: NTU
    long_name: constant turbidity
unlimited dimensions: time_counter
current shape = (1, 40, 10, 30)
filling on, default _FillValue of 9.969209968386869e+36 used

In [16]:
#turStep
turStepN = newN.createVariable('turStep', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
turStepN.grid = tN.variables['NO3'].grid
turStepN.units = 'NTU'
turStepN.long_name = 'constant turbidity' 
turStepN[:]=1.0

turStepW = newW.createVariable('turStep', 'float32', 
                               ('time_counter','deptht','yb','xbT'))
turStepW.grid = tW.variables['NO3'].grid
turStepW.units = 'NTU'
turStepW.long_name = 'constant turbidity' 
turStepW[:]=1.0
turStepW[:,10:,:,:]=.1

In [17]:
turStepW[0,:,0,0]


masked_array(data=[1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.1,
                   0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
                   0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
                   0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [18]:
newW.close()
newN.close()
tN.close()
tW.close()

In [ ]:
# load database for data-based conditions

In [ ]:
# definitions
basepath='/ocean/eolson/MEOPAR/obs/'
basedir=basepath + 'DFOOPDB/'
dbname='DFO_OcProfDB'

# engine and reflection
Base = automap_base()
engine = create_engine('sqlite:///' + basedir + dbname + '.sqlite', echo = False)
Base.prepare(engine, reflect=True)
Station=Base.classes.StationTBL
Obs=Base.classes.ObsTBL
JDFLocs=Base.classes.JDFLocsTBL
Calcs=Base.classes.CalcsTBL
session = create_session(bind = engine, autocommit = False, autoflush = True)

# column definitions
SA=case([(Calcs.Salinity_Bottle_SA!=None, Calcs.Salinity_Bottle_SA)], else_=
         case([(Calcs.Salinity_T0_C0_SA!=None, Calcs.Salinity_T0_C0_SA)], else_=
         case([(Calcs.Salinity_T1_C1_SA!=None, Calcs.Salinity_T1_C1_SA)], else_=
         case([(Calcs.Salinity_SA!=None, Calcs.Salinity_SA)], else_=
         case([(Calcs.Salinity__Unknown_SA!=None, Calcs.Salinity__Unknown_SA)], else_=Calcs.Salinity__Pre1978_SA)
        ))))
NO=case([(Obs.Nitrate_plus_Nitrite!=None, Obs.Nitrate_plus_Nitrite)], else_=Obs.Nitrate)
NOUnits=case([(Obs.Nitrate_plus_Nitrite!=None, Obs.Nitrate_plus_Nitrite_units)], else_=Obs.Nitrate_units)
NOFlag=case([(Obs.Nitrate_plus_Nitrite!=None, Obs.Flag_Nitrate_plus_Nitrite)], else_=Obs.Flag_Nitrate)
# Obs.Quality_Flag_Nitr does not match any nitrate obs
# ISUS not included in this NO
Tem=case([(Obs.Temperature!=None, Obs.Temperature)], else_=
         case([(Obs.Temperature_Primary!=None, Obs.Temperature_Primary)], else_=
         case([(Obs.Temperature_Secondary!=None, Obs.Temperature_Secondary)], else_=Obs.Temperature_Reversing)))
TemUnits=case([(Obs.Temperature!=None, Obs.Temperature_units)], else_=
         case([(Obs.Temperature_Primary!=None, Obs.Temperature_Primary_units)], else_=
         case([(Obs.Temperature_Secondary!=None, Obs.Temperature_Secondary_units)], 
              else_=Obs.Temperature_Reversing_units)))
TemFlag=Obs.Quality_Flag_Temp
Ox=case([(Calcs.Oxygen_umolL!=None, Calcs.Oxygen_umolL)], else_=Calcs.Oxygen_Dissolved_umolL)
OxFlag=case([(Calcs.Oxygen_umolL!=None, Obs.Quality_Flag_Oxyg)], else_=Obs.Flag_Oxygen_Dissolved)
Press=case([(Obs.Pressure!=None, Obs.Pressure)], else_=Obs.Pressure_Reversing)

In [ ]:
# Ammonium:

In [ ]:
q=session.query(JDFLocs.ObsID, Station.StartYear,Station.StartMonth,Press,
                Obs.Ammonium,Obs.Ammonium_units,Tem,SA).select_from(Obs).\
        join(JDFLocs,JDFLocs.ObsID==Obs.ID).join(Station,Station.ID==Obs.StationTBLID).\
        join(Calcs,Calcs.ObsID==Obs.ID).filter(Obs.Ammonium!=None).\
        all()
qP=[]
qNH=[]
remP=[]
remNH=[]
for OID, Yr, Mn, P, NH, un, T, S_A in q:
    # throw out 1 data point that seems unusually high
    if not (P>75 and NH >.2):
        qP.append(P)
        qNH.append(NH)
    else:
        remP.append(P)
        remNH.append(NH)
qP=np.array(qP)
qNH=np.array(qNH)
remP=np.array(remP)
remNH=np.array(remNH)

In [ ]:
len(qP), len(qNH), np.max(qP),np.min(qP),np.max(qNH),np.min(qNH)

In [ ]:
# create depth-weighted mean profile using gaussian filter
zs=np.array(TS.variables['deptht'])
AmmProf=ginterp(qP,0.0,qNH,10,zs)
AmmProf[AmmProf!=AmmProf]=0.0


In [ ]:
for ii in range(0,zs.size):
    voNH4[:,ii,0,:]=AmmProf[ii]

In [ ]:
# DON

In [ ]:
# take nearest available data to SJDF
q=session.query(Station.StartYear,Station.StartMonth,Press, Station.Lat, Station.Lon,Obs.Depth,
                Obs.Nitrogen_Dissolved_Organic,Obs.Nitrogen_Dissolved_Organic_units,Tem).\
        select_from(Obs).join(Station,Station.ID==Obs.StationTBLID).\
        filter(Obs.Nitrogen_Dissolved_Organic!=None).filter(Obs.Nitrogen_Dissolved_Organic>=0).\
        filter(Station.Lat!=None).filter(Station.Lon!=None).\
        filter(Station.Lat<48.8).filter(Station.Lon<-125).all()

qDON=[]
for row in q:
    qDON.append(row[6])
val_DON=np.mean(qDON)

In [ ]:
voDON[:,:,:,:]=val_DON

In [ ]:
# PON

# take nearest available data to SJDF
q=session.query(Station.StartYear,Station.StartMonth,Press, Station.Lat, Station.Lon,Obs.Depth,
                Obs.Nitrogen_Particulate_Organic,Obs.Nitrogen_Particulate_Organic_units,Tem).\
        select_from(Obs).join(Station,Station.ID==Obs.StationTBLID).\
        filter(Obs.Nitrogen_Particulate_Organic!=None).filter(Obs.Nitrogen_Particulate_Organic>=0).\
        filter(Station.Lat!=None).filter(Station.Lon!=None).\
        filter(Station.Lat<48.8).filter(Station.Lon<-125).all()

qPON=[]
for row in q:
    qPON.append(row[6])
val_PON=np.mean(qPON)

voPON[:,:,:,:]=val_PON

newConst.close()
TS.close()

set up NO3 and save climatology:

In [ ]:
# umol/L=mmol/m**3, so all NO units the same
q=session.query(JDFLocs.ObsID, Station.StartYear,Station.StartMonth,Press,NO,Tem,SA,Station.StartDay).select_from(Obs).\
        join(JDFLocs,JDFLocs.ObsID==Obs.ID).join(Station,Station.ID==Obs.StationTBLID).\
        join(Calcs,Calcs.ObsID==Obs.ID).filter(SA<38).filter(SA>0).filter(NO!=None).\
        filter(Tem!=None).filter(SA!=None).filter(Press!=None).\
        all()
#for row in q:
#    print(row)

qYr=[]
qMn=[]
qDy=[]
qP=[]
qNO=[]
qTC=[]
qSA=[]
qNO50=[]
qSA50=[]
qTC50=[]
date=[]
for OID, Yr, Mn, P, NO3, T, S_A, dy in q:
    qYr.append(Yr)
    qMn.append(Mn)
    qDy.append(dy)
    qP.append(P)
    qNO.append(NO3)
    qTC.append(gsw.CT_from_t(S_A,T,P))
    qSA.append(S_A)
    date.append(dt.date(int(Yr),int(Mn),int(dy)))
    if P>80:
        qNO50.append(NO3)
        qTC50.append(gsw.CT_from_t(S_A,T,P))
        qSA50.append(S_A)

qSA=np.array(qSA)
qTC=np.array(qTC)
qP=np.array(qP)
qNO=np.array(qNO)
qSA50=np.array(qSA50)
qTC50=np.array(qTC50)
date=np.array(date)
YD=0.0*qTC
for i in range(0,len(YD)):
    YD[i]=date[i].timetuple().tm_yday

qNO50=np.array(qNO50)

In [ ]:
print(qTC50)

In [ ]:
a=np.vstack([qTC50,qSA50,np.ones(len(qTC50))]).T
a2=np.vstack([qTC,qSA,np.ones(len(qTC))]).T
m = np.linalg.lstsq(a,qNO50)[0]
mT, mS, mC = m

zupper=np.extract(zs<100, zs)
ydays=np.arange(0,365,365/52)

In [ ]:
# umol/L=mmol/m**3, so all NO units the same
q=session.query(JDFLocs.ObsID, Station.StartYear,Station.StartMonth,Press,NO,Tem,SA,Station.StartDay).select_from(Obs).\
        join(JDFLocs,JDFLocs.ObsID==Obs.ID).join(Station,Station.ID==Obs.StationTBLID).\
        join(Calcs,Calcs.ObsID==Obs.ID).filter(SA<38).filter(SA>0).filter(NO!=None).\
        filter(Tem!=None).filter(SA!=None).filter(Press<120).filter(Press!=None).\
        all()
#for row in q:
#    print(row)

qYr=[]
qMn=[]
qDy=[]
qP=[]
qNO=[]
qTC=[]
qSA=[]
date=[]
for OID, Yr, Mn, P, NO3, T, S_A, dy in q:
    qYr.append(Yr)
    qMn.append(Mn)
    qDy.append(dy)
    qP.append(P)
    qNO.append(NO3)
    qTC.append(gsw.CT_from_t(S_A,T,P))
    qSA.append(S_A)
    date.append(dt.date(int(Yr),int(Mn),int(dy)))

qSA=np.array(qSA)
qTC=np.array(qTC)
qP=np.array(qP)
qNO=np.array(qNO)
date=np.array(date)
YD=0.0*qTC
for i in range(0,len(YD)):
    YD[i]=date[i].timetuple().tm_yday




In [ ]:
mC,mT,mS

In [ ]:
ndict,nmat=ginterp2d(YD,365,qP,0,qNO,30,10,ydays,zupper)
np.savetxt('/home/eolson/pyCode/notebooks/modelInput/OBC/nmatTest.csv',nmat,delimiter=',')

df=pd.DataFrame({'mC':[mC],'mT':[mT],'mS':[mS]})
df.to_csv('/home/eolson/pyCode/notebooks/modelInput/OBC/bioOBCfit_NTSTest.csv')


set up Si and save climatology:

In [ ]:
# umol/L=mmol/m**3, so all NO units the same
q=session.query(JDFLocs.ObsID, Station.StartYear,Station.StartMonth,Press,Obs.Silicate,Tem,SA,Station.StartDay).select_from(Obs).\
        join(JDFLocs,JDFLocs.ObsID==Obs.ID).join(Station,Station.ID==Obs.StationTBLID).\
        join(Calcs,Calcs.ObsID==Obs.ID).filter(SA<38).filter(SA>0).filter(Obs.Silicate!=None).\
        filter(Tem!=None).filter(SA!=None).filter(Press!=None).\
        all()
#for row in q:
#    print(row)

qYr=[]
qMn=[]
qDy=[]
qP=[]
qNO=[]
qTC=[]
qSA=[]
qNO50=[]
qSA50=[]
qTC50=[]
date=[]
for OID, Yr, Mn, P, NO3, T, S_A, dy in q:
    qYr.append(Yr)
    qMn.append(Mn)
    qDy.append(dy)
    qP.append(P)
    qNO.append(NO3)
    qTC.append(gsw.CT_from_t(S_A,T,P))
    qSA.append(S_A)
    date.append(dt.date(int(Yr),int(Mn),int(dy)))
    if P>80:
        qNO50.append(NO3)
        qTC50.append(gsw.CT_from_t(S_A,T,P))
        qSA50.append(S_A)

qSA=np.array(qSA)
qTC=np.array(qTC)
qP=np.array(qP)
qNO=np.array(qNO)
qSA50=np.array(qSA50)
qTC50=np.array(qTC50)
date=np.array(date)
YD=0.0*qTC
for i in range(0,len(YD)):
    YD[i]=date[i].timetuple().tm_yday

qNO50=np.array(qNO50)


In [ ]:
a=np.vstack([qTC50,qSA50,np.ones(len(qTC50))]).T
a2=np.vstack([qTC,qSA,np.ones(len(qTC))]).T
m = np.linalg.lstsq(a,qNO50)[0]
mT, mS, mC = m
print(mT, mS, mC)

In [ ]:
# umol/L=mmol/m**3, so all NO units the same
q=session.query(JDFLocs.ObsID, Station.StartYear,Station.StartMonth,Press,Obs.Silicate,Tem,SA,Station.StartDay).select_from(Obs).\
        join(JDFLocs,JDFLocs.ObsID==Obs.ID).join(Station,Station.ID==Obs.StationTBLID).\
        join(Calcs,Calcs.ObsID==Obs.ID).filter(SA<38).filter(SA>0).filter(Obs.Silicate!=None).\
        filter(Tem!=None).filter(SA!=None).filter(Press<120).filter(Press!=None).\
        all()
#for row in q:
#    print(row)

qYr=[]
qMn=[]
qDy=[]
qP=[]
qNO=[]
qTC=[]
qSA=[]
date=[]
for OID, Yr, Mn, P, NO3, T, S_A, dy in q:
    qYr.append(Yr)
    qMn.append(Mn)
    qDy.append(dy)
    qP.append(P)
    qNO.append(NO3)
    qTC.append(gsw.CT_from_t(S_A,T,P))
    qSA.append(S_A)
    date.append(dt.date(int(Yr),int(Mn),int(dy)))

qSA=np.array(qSA)
qTC=np.array(qTC)
qP=np.array(qP)
qNO=np.array(qNO)
date=np.array(date)
YD=0.0*qTC
for i in range(0,len(YD)):
    YD[i]=date[i].timetuple().tm_yday



In [ ]:
sidict,simat=ginterp2d(YD,365,qP,0,qNO,30,10,ydays,zupper)

np.savetxt('/home/eolson/pyCode/notebooks/modelInput/OBC/simatTest.csv',simat,delimiter=',')

df=pd.DataFrame({'mC':[mC],'mT':[mT],'mS':[mS]})
df.to_csv('/home/eolson/pyCode/notebooks/modelInput/OBC/bioOBCfit_SiTSTest.csv')